In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.5
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
import pyspark.sql.functions as f
from pyspark.ml.linalg import DenseVector, SparseVector
from pyspark.ml.feature import StopWordsRemover, Tokenizer, HashingTF, IDF
from pyspark.ml import Pipeline

In [3]:
data = spark.read.json('/labs/laba02/DO_record_per_line.json')

In [4]:
print(data.count())
data = data.filter(f.length("desc") > 50)
print(data.count())

28153
27878


In [5]:
tokenizer = Tokenizer(inputCol="desc", outputCol="words")
hashtf = HashingTF(numFeatures=10000, inputCol="words", outputCol='tf')
idf = IDF(inputCol='tf', outputCol="features", minDocFreq=5)

pipeline = Pipeline(stages=[tokenizer, hashtf, idf])
pipe = pipeline.fit(data)
data2 = pipe.transform(data).cache()

In [6]:
target_films = [23126, 21617, 16627, 11556, 16704, 13702]
targets = data2.filter(f.col('id').isin(target_films)).select('id', 'features').collect()
df = data2.select('id', 'features').collect()

In [7]:
%%time
all_ = []
sim = {}
for i in range(len(targets)):
    u = targets[i][1]
    si = {}
    for k in range(len(df)):
        v = df[k][1]
        si[df[k][0]] =  v.dot(u) / (v.norm(2) * u.norm(2))
    sim[targets[i][0]] = si
    print('done')
    all_.append(sim)

done
done
done
done
done
done
CPU times: user 10.3 s, sys: 4 ms, total: 10.3 s
Wall time: 10.4 s


In [8]:
rec=[]
sims = {}
for i, t in zip(range(len(sim)), target_films) :
    sorted_ = sorted(sim[t].items(), key=lambda item: item[1], reverse=True)[:11]
    rec = []
    for k in range(len(sorted_)):
        rec.append(sorted_[k][0])
    sims[t] = rec
    
for i in target_films:
    sims[i].remove(i)

In [9]:
sims

{23126: [13782, 13665, 24419, 20638, 2724, 25782, 2633, 2723, 13348, 15909],
 21617: [21609, 21608, 21616, 21492, 21624, 21623, 21630, 21628, 21508, 21857],
 16627: [11431, 12247, 13021, 25010, 11575, 5687, 9598, 5372, 12863, 16769],
 11556: [16488, 13461, 468, 10447, 387, 22710, 9289, 5936, 23357, 7833],
 16704: [3864, 23407, 1365, 20645, 1426, 20105, 8217, 1236, 1164, 23864],
 13702: [864, 1216, 7173, 8313, 1052, 17017, 19613, 21017, 17015, 8082]}

In [10]:
import json

with open("lab02.json", "w", encoding="utf-8") as file:
    json.dump(sims, file)

In [11]:
sc.stop()